In [ ]:
#均值填补
from sklearn.impute import SimpleImputer

#missinf_values替换的值
imp_mean = SimpleImputer(missinf_values=np.nan,strategy="mean")

#X_missing_data处理的数据
X_missing_mean = imp_mean.fit_transform(X_missing_data)

In [ ]:
#数值填补

#missinf_values替换的值
imp_0 = SimpleImputer(missinf_values=np.nan,strategy="constant",fill_value=0)

#X_missing_data处理的数据
X_missing_0 = imp_0.fit_transform(X_missing_data)

In [ ]:
#随机森林填补

X_missing_reg = X_missing_data.copy()

#缺失特征排序，先补缺失少的（返回对应缺失特征索引）
sortindex = np.argsort(X_missing_reg.isnull().sum(axis=0)).values

#将非本次填充特征补0
for i in sortindex:
    
    df = X_missing_reg

    #新标签
    fillc = df.iloc[:,i]

    #新矩阵
    df = pd.concat([df.iloc[: df.columns!=i],pd.DataFrame(y_full)],axis=1)

    #补0
    df_0 = SimpleImputer(missinf_values=np.nan,strategy="constant",fill_value=0).fit_transform(df)

    #定义训练集测试集
    Ytrain = fillc[fillc.notnull()]#非空的为训练集的Y，根据索引获取
    Ytest = fillc[fillc.isnull()]#给Xtest提供索引

    Xtrain = df_0[Ytrain.index,:]#根据索引获取训练集，填0之后的
    Xtest = df_0[Ytest.index,:]#需要填充的缺失项

    rfc = RandomForestRegressor(n_estimators=100)
    rfc = rfc.fit(Xtrain,Ytrain)

    Ypredict = rfc.predict(Xtest)#得到结果

    #填补
    X_missing_reg_.loc[X_missing_reg.iloc[:,i].isnull(),i] = Ypredict